# Module 1 - Programming Assignment


In [356]:
import copy
import sys

# State Space Search with A* Search

This program is an implementation of the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
#       hills      5
~       swamp      7
x       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
....*..
...***.
.###...
..##...
..#..**
....***
.......
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [357]:
full_world = [
  ['.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '#', 'x', 'x', '#', '#'], 
  ['.', '.', '.', '.', '#', 'x', 'x', 'x', '*', '*', '*', '*', '~', '~', '*', '*', '*', '*', '*', '.', '.', '#', '#', 'x', 'x', '#', '.'], 
  ['.', '.', '.', '#', '#', 'x', 'x', '*', '*', '.', '.', '~', '~', '~', '~', '*', '*', '*', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.'], 
  ['.', '#', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '~', '~', '~', '~', '~', '.', '.', '.', '.', '.', '#', 'x', '#', '.', '.'], 
  ['.', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '.', '.', '.', '#', '.', '.', '.'], 
  ['.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '#', '#', '#', '.', '.'], 
  ['.', '.', '.', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '.', '~', '~', '.', '.', '#', '#', '#', '.', '.', '.'], 
  ['.', '.', '.', '~', '~', '~', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '~', '.', '#', '#', '#', '.', '.', '.', '.'], 
  ['.', '.', '~', '~', '~', '~', '~', '.', '#', '#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.', '.', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['~', '~', '~', '~', '~', '.', '.', '#', '#', 'x', 'x', '#', '.', '~', '~', '~', '~', '.', '.', '.', '#', 'x', '#', '.', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '.', '.', '#', '*', '*', '#', '.', '.', '.', '.', '~', '~', '~', '~', '.', '.', '#', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', '.', '.', '*', '*', '*', '*', '#', '#', '#', '#', '.', '~', '~', '~', '.', '.', '#', 'x', '#', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '.', '~', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '.', '.', 'x', 'x', 'x', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '#', '#', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~'], 
  ['.', '.', '#', '#', '#', '#', 'x', 'x', '*', '*', '*', '*', '*', '.', 'x', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', '*', '*', 'x', 'x', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '#', '#', '.', '.', '~', '~', '~', '~', '~', '~'], 
  ['.', '#', '#', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '#', '#', '.', '~', '~', '~', '~', '~'], 
  ['#', 'x', '#', '#', '#', '#', '.', '.', '.', '.', '.', 'x', 'x', 'x', '#', '#', 'x', 'x', '.', 'x', 'x', '#', '#', '~', '~', '~', '~'], 
  ['#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', 'x', 'x', '#', '#', '#', '#', 'x', 'x', 'x', '~', '~', '~', '~'], 
  ['#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '#', '#', '#', '.', '.', '.']]

<div style="background: khaki; color: darkgoldenrod; border: 2px solid goldenrod; padding: 5px; margin: 10px;">
<strong>Warning</strong>
</div>

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7).

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics, what do you expect the policy (plan or path to the goal) would be?

In [358]:
test_world = [
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
]

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.  Moves: the legal movement model expressed in offsets.

In [359]:
cardinal_moves = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [360]:
costs = { '.': 1, '*': 3, '#': 5, '~': 7}

## A\* Search Implementation

### Helper Methods

-----
**Get the cost of a state**

Getting the cost of each move to a new state is essential for the A\* algorithm.  The get_cost function takes a state ((x,y) tuple) and a world (list of lists). It finds the location in the world in order to determine the terrain at that location. The costs `Dict` is then used to determine the cost of that terrain, which is returned.

In [361]:
def get_cost(state, world):
    x = state[0]
    y = state[1]
    state_type = world[x][y]
    
    cost = costs[state_type]
    
    return cost

-----
**Get the cost so far for a state**

The A\* algorithm requires keeping track of the cost so far of each state along the path in order to calculate f(n) = g(n) + h(n), where g(n) is the cost so far, and h(n) is the heuristic.  To calculate the cost so far of a state, get the cost to enter that state using the get_cost function. Then add the state's cost to the cost so far of its parent state.  A dictionary of states is used to keep track of (x,y): [f(n), cost so far, parent state] for each state.  This way information can be easily looked up about each state.

In [362]:
def get_cost_so_far(state, previous_state, world, states):
    if previous_state == None:
        previous_cost = 0
    else:
        previous_cost = states[previous_state][1]
    
    cost = get_cost(state, world) + previous_cost
    
    return cost
    

-----
**Get f(n)**

As stated above, f(n) is essential for the A\* algorithm, and determines the ordering of our priority queue that keeps track of the frontier as we move through the world.  The get_fn function gets the cost so far of the given state, using the get_cost_so_far function.  It adds this value to the output of the heuristic function, giving an estimate of the total cost from the start to the goal through the current node.


In [363]:
def get_fn(heuristic, state, previous_state, world, goal, states):
    return get_cost_so_far(state, previous_state, world, states) + heuristic(state, goal)

-----
**Insert an item into the frontier**

The frontier is a priority queue, ordered by f(n).  We use the heuristic function, state, previous state, world, goal, and states dictionary to get f(n) using the get_fn function, as described above.  We then interate over the frontier to find the correct placement of the state in the frontier.  If the current state is already in the frontier with a higher f(n), the state's location is updated with the new f(n).  This function also updates cost so far and parent state in the states dictionary.

In [364]:
def insert_into_frontier(heuristic, state, previous_state, world, goal, frontier, states):
    fn = get_fn(heuristic, state, previous_state, world, goal, states)
    
    if state in frontier:
        existing_fn = states[state][0]
        if existing_fn < fn:
            return
        else: 
            frontier.remove(state)
    
    for i in range(len(frontier)):
        existing_state = frontier[i]
        existing_fn = states[existing_state][0]
        
        if fn < existing_fn and state not in frontier:
            frontier.insert(i, state)
    
    if state not in frontier:
        frontier.append(state)
        
    cost_so_far = get_cost_so_far(state, previous_state, world, states)
    states[state] = [fn, cost_so_far, previous_state]

-----
**Determine whether a state is valid**

The function is_valid determine whether the state is valid to be moved to by the agent in given the world.  A state is valid if it is within the bounds of the world, so, the its x, y coordinates are not outside the indices of the list of lists that makes up the world.  A state is not valid to be moved to if it is mountains terrain, specifically if it is the character 'x'. This function is necessary so that at each step through the world, the algorithm can determine which steps it can take next.


In [365]:
def is_valid(state, world):
    x = state[0]
    y = state[1]
    
    world_depth = len(world)
    if x < 0 or x >= world_depth:
        return False
        
    world_width = len(world[x])
    if y < 0 or y >= world_width:
        return False
    
    if world[x][y] == 'x':
        return False
    
    return True


-----
**Get the next moves available**

The function get_successors is given a state and a world and gets the next available moves based on the given movement model.  It uses the possible moves to find all coordinates that would result from those moves.  It then checks each of the coordinates to make sure they are valid moves using the is_valid function.

In [366]:
def get_successors(state, world, moves):
    next_moves = []
        
    for move in moves:
        x = state[0] + move[1]
        y = state[1] + move[0]
        
        new_state = (x,y)
        
        if is_valid(new_state, world):
            next_moves.append(new_state)
    
    return next_moves

-----
**Construct the path**

This function finds the path taken by the agent to get from the start to the goal.  Given the goal and the list of states that were visited in finding the optimal path, construct_path starts at the goal and works backward using the previous state stored for each state in the states dictionary.  Previous state is at index 2 in the list that each (x,y) key points to in the dictionary.  The function first finds a list of the path coordinates, then uses these coordinates to construct the path to be returned.  This path is the offsets needed to get from start state to the goal as a list.  The function uses the difference in x and y coordinates between each step to get these offsets.

In [367]:
def construct_path(goal, states):
    path_coordinates = []
    path = []
    current = goal
    
    while current:
        path_coordinates.insert(0, current)
        current = states[current][2]

    num_steps = len(path_coordinates)
    for i in range(num_steps):
        current_point = path_coordinates[i]
        
        if i < num_steps - 1:
            next_point = path_coordinates[i + 1]
        
            xDiff = next_point[1] - current_point[1]
            yDiff = next_point[0] - current_point[0]
        
            path.append((xDiff, yDiff))   
           
    return path

-----
**Print the world**

This function prints the world as a grid with no spaces or extra characters.  It is used to pretty print the final solution as a path through the world using characters '>', '<', '^', 'v' to show the directions that the agent traveled.

In [368]:
def print_world(world):
    for row in world:
        for col in row:
            sys.stdout.write(col)
        print

-----
**Get the symbol that represents movement in the world**

As described above, characters '>', '<', '^', 'v' represent the directions the agent traveled in the world.  Given a step from the path list that contains the offsets needed to get from start state to the goal, this function returns the character that indicates which direction was taken.

In [369]:
def get_path_symbol(step):
    if step[0] > 0:
        return '>'
    
    if step[0] < 0:
        return '<'
    
    if step[1] > 0:
        return 'v'
        
    if step[1] < 0:
        return '^'    



-----

**a_star_search**

The `a_star_search` function uses the A\* Search algorithm to solve a navigational problem for an agent in a grid world. It calculates a path from the start state to the goal state and returns the actions required to get from the start to the goal.

* **world** is the starting state representation for a navigation problem.
* **start** is the starting location, `(x, y)`.
* **goal** is the desired end position, `(x, y)`.
* **costs** is a `Dict` of costs for each type of terrain.
* **moves** is the legal movement model expressed in offsets.
* **heuristic** is a heuristic function that returns an estimate of the total cost $f(x)$ from the start to the goal through the current node, $x$. The heuristic function might change with the movement model.


The function returns the offsets needed to get from start state to the goal as a `List`. For example, for the test world:

```
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],

```

it would return:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

Do not make unwarranted assumptions. For example, do not assume the starting point is always `(0, 0)` or that the goal is always in the lower right hand corner. Do not make any assumptions about the movement model beyond the requirement that they be offsets (it could be offets of 2!).

In [370]:
def a_star_search(world, start, goal, costs, moves, heuristic):
    frontier = []
    explored = []
    path = []
    start = (start[1], start[0])
    goal = (goal[1], goal[0])
    states = { start: [0, heuristic(start, goal), None] }
    
    frontier.append(start)

    while frontier: 
        current_state = frontier.pop(0) 
        if current_state == goal:
            return construct_path(current_state, states)
        
        children = get_successors(current_state, world, moves)
        for child in children:
            if child not in explored:
                insert_into_frontier(heuristic, child, current_state, world, goal, frontier, states)  #this needs to check if already in frontier
        
        explored.append(current_state)
    
    return []

**pretty_print_solution**

The `pretty_print_solution` function prints an ASCII representation of the solution generated by the `a_star_search`. For example, for the test world, it would take the `world` and `path` and print:

```
v******
v******
v******
>>>>>>v
******v
******v
******G
```

using `v`, `^`, `>`, `<` to represent actions and `G` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters).


In [371]:
def pretty_print_solution(world, path, start):
    start = (start[1], start[0])
    world_copy = copy.deepcopy(world)
    world_copy[start[0]]
    current_location = start
    
    for step in path:
        symbol = get_path_symbol(step)
        
        world_copy[current_location[0]][current_location[1]] = symbol
        
        current_location0 = current_location[0] + step[1]
        current_location1 = current_location[1] + step[0]
        
        current_location = (current_location0, current_location1)
        
    world_copy[current_location[0]][current_location[1]] = 'G'   
    print_world(world_copy)

Execute `a_star_search` and `print_path` for the `test_world` and the `real_world`.

**Heuristic function**

This heuristic function finds the manhattan distance from the given state to the goal state, assuming a cost of 1.  I assume a cost of 1, to ensure the heuristic is admissible, e.g. does not overestimate the cost to reach the goal.

In [372]:
# heuristic function
def heuristic(state, goal):
    x = state[0]
    y = state[1]
    
    xDifference = abs(x - goal[0])
    yDifference = abs(y - goal[1])
    return xDifference + yDifference

In [373]:
test_path = a_star_search( test_world, (0, 0), (6, 6), costs, cardinal_moves, heuristic)
print test_path

[(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]


In [374]:
pretty_print_solution( test_world, test_path, (0, 0))

v******
v******
v******
>>>>>>v
******v
******v
******G


In [375]:
full_path = a_star_search( full_world, (0, 0), (26, 26), costs, cardinal_moves, heuristic)
print full_path

[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (0, 1), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0)]


In [376]:
pretty_print_solution( full_world, full_path, (0, 0))

v....**********............
v......*********..xxxxxxx..
v...xx***********xxx###xx##
v...#xxx****~~*****..##xx#.
v..##xx**..~~~~***...#xxx#.
v###xx##....~~~~~.....#x#..
v##xx##....#xxx~~~.....#...
v.#####......#xxx~~~..###..
v..###......##xx.~~..###...
v..~~~..###xxxx...~.###....
v.~~~~~.##xxx#.....#xxx#...
v~~~~~..#xx#....~~..#xx#...
v~~~~..##xx#.~~~~...#x#....
v~~~~..#**#....~~~~..#.....
v...x..****####.~~~..#x#...
v..xxx******xxx##.~.#xx#...
v.xx**********xxx..xxx.....
v..xx***********xxxx.......
v..xxx********...##........
v...xxx******..........~~~~
v.####xx*****.x.....~~~~~~~
v...###xxx**xx......~~~~~~~
>>>v..###xxxx....##..~~~~~~
.##>v#####.....##xx##.~~~~~
#x##v#.....xxx##xx.xx##~~~~
#xxxv.....##xxxx####xxx~~~~
##..>>>>>>>>>>>>>>>>>>>>>>G
